<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name: Andrew Pickner

<br>




**Some things that might be useful**


In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
'''
Copied and pasted from in-class notebook: CSCI3202_TicTacToe_AlphaBeta_Solutions

2/21/2020
'''

# Fill in as needed.
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)
    
    def update_state(self, move):
        if self.to_move == 'X':
            self.board[move] = self.to_move
            self.moves.pop(self.moves.index(move))
            self.to_move = 'O'
        else:
            self.board[move] = self.to_move
            self.moves.pop(self.moves.index(move))
            self.to_move = 'X'
        
class TicTacToe:
    def __init__(self, nrow=3, ncol=3, nwin=3):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''
        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state

    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        
        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''
        return state.utility!=0 or len(state.moves)==0    

    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        return state.utility if player=='X' else -state.utility

    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''
        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
                    self.display()
                    return self.state.utility   

<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
'''
Copied and pasted from in-class notebook: CSCI3202_TicTacToe_AlphaBeta_Solutions

2/21/2020
'''
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''
    # your code goes here...
    possible_actions = game.state.moves
    # np.random.randint
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]

<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [4]:
num_games = 1000

first_player_wins = 0
draws = 0
second_player_wins = 0
for i in range(1, num_games+1):
    print('Game #{}'.format(i))
    ttt = TicTacToe(3,3,3)
    outcome = ttt.play_game(random_player, random_player)
    if outcome == 1:
        first_player_wins += 1
        print('P1 wins')
    elif outcome == 0:
        draws += 1
        print('DRAW')
    else:
        second_player_wins += 1
        print('P2 wins')

print("P1 wins: {}".format(float(first_player_wins) / float(num_games)))
print("Draws: {}".format(float(draws) / float(num_games)))
print("P2 wins: {}".format(float(second_player_wins) / float(num_games)))

Game #1
X . O 
X X . 
O O X 
P1 wins
Game #2
O O X 
X X O 
O X X 
DRAW
Game #3
O O X 
X X O 
X X O 
P1 wins
Game #4
X O O 
X X X 
O X O 
P1 wins
Game #5
O O X 
. X X 
O . X 
P1 wins
Game #6
X . O 
. O X 
O X . 
P2 wins
Game #7
X X . 
O O O 
X O X 
P2 wins
Game #8
O X X 
. O X 
X O O 
P2 wins
Game #9
O X X 
X X O 
X O O 
P1 wins
Game #10
X X X 
X O O 
O X O 
P1 wins
Game #11
X O O 
O X X 
X X O 
DRAW
Game #12
O . X 
O O X 
. X X 
P1 wins
Game #13
O O O 
O X X 
. X X 
P2 wins
Game #14
. X O 
O X X 
. X O 
P1 wins
Game #15
X O . 
. O O 
X X X 
P1 wins
Game #16
O X O 
X X X 
X O O 
P1 wins
Game #17
X . O 
X O X 
O . . 
P2 wins
Game #18
X O X 
O X X 
X O O 
P1 wins
Game #19
X X O 
O O X 
X O X 
DRAW
Game #20
O X . 
. X . 
O X . 
P1 wins
Game #21
O O X 
X X O 
X O X 
P1 wins
Game #22
. O O 
X X X 
. X O 
P1 wins
Game #23
X X . 
O O O 
. . X 
P2 wins
Game #24
X O O 
X X . 
X . O 
P1 wins
Game #25
X . . 
O X X 
O O X 
P1 wins
Game #26
O . O 
O X X 
O X X 
P2 wins
Game #27
X O X 
X O X 
O X O 


X O O 
X . . 
X . . 
P1 wins
Game #247
X O X 
X X O 
O O X 
P1 wins
Game #248
O O O 
X X O 
X X . 
P2 wins
Game #249
X X O 
O X X 
X O O 
DRAW
Game #250
X O X 
X X O 
O O X 
P1 wins
Game #251
. . O 
. X O 
X X O 
P2 wins
Game #252
X O O 
X X X 
O O X 
P1 wins
Game #253
O X O 
X O X 
X O X 
DRAW
Game #254
O X . 
O . O 
X X X 
P1 wins
Game #255
X O X 
O X O 
X X O 
P1 wins
Game #256
O X X 
O X X 
O O . 
P2 wins
Game #257
O X X 
O O X 
X O X 
P1 wins
Game #258
. O O 
. . . 
X X X 
P1 wins
Game #259
O O X 
O X X 
. . X 
P1 wins
Game #260
O O X 
O X X 
. . X 
P1 wins
Game #261
X X O 
O X O 
X O X 
P1 wins
Game #262
X O . 
X O X 
X . O 
P1 wins
Game #263
O X O 
O X X 
X X O 
P1 wins
Game #264
X O X 
O X X 
X O O 
P1 wins
Game #265
X X . 
X X O 
O O O 
P2 wins
Game #266
O O X 
X X O 
O X X 
DRAW
Game #267
X O O 
. X . 
X O X 
P1 wins
Game #268
X X O 
X O X 
X O O 
P1 wins
Game #269
X X O 
O X O 
O X X 
P1 wins
Game #270
X O . 
O O . 
X X X 
P1 wins
Game #271
. O X 
X O . 
. O X 
P2 wins
Game 

X X X 
X O O 
O X O 
P1 wins
Game #514
X X X 
. X O 
O O . 
P1 wins
Game #515
X X O 
O O X 
X O X 
DRAW
Game #516
. O X 
. X O 
X O X 
P1 wins
Game #517
. X O 
. . O 
X X O 
P2 wins
Game #518
X X O 
. X . 
O X O 
P1 wins
Game #519
O O X 
X X O 
O X X 
DRAW
Game #520
X O O 
X X X 
O X O 
P1 wins
Game #521
. O X 
. O X 
. . X 
P1 wins
Game #522
X O . 
O O X 
X O X 
P2 wins
Game #523
X O X 
O X X 
O X O 
DRAW
Game #524
. X X 
O X X 
O O O 
P2 wins
Game #525
X X . 
. X . 
O O O 
P2 wins
Game #526
X O X 
X O O 
X X O 
P1 wins
Game #527
X O O 
O X X 
X O X 
P1 wins
Game #528
X O X 
X O O 
. O X 
P2 wins
Game #529
X . O 
X . O 
X . . 
P1 wins
Game #530
O X X 
X O O 
X O X 
DRAW
Game #531
O O X 
. X X 
. O X 
P1 wins
Game #532
O X O 
X X X 
X O O 
P1 wins
Game #533
X O O 
X X X 
O O X 
P1 wins
Game #534
O . X 
. X . 
X . O 
P1 wins
Game #535
X X X 
. . O 
O O X 
P1 wins
Game #536
O X X 
. O X 
O X O 
P2 wins
Game #537
. O O 
X X O 
X X O 
P2 wins
Game #538
X O . 
. O O 
X X X 
P1 wins
Game #53

O X . 
X X . 
P1 wins
Game #750
X X O 
O O X 
X X O 
DRAW
Game #751
X O O 
O O X 
X X X 
P1 wins
Game #752
O X O 
X O O 
X X X 
P1 wins
Game #753
O O X 
X X O 
X O X 
P1 wins
Game #754
X O X 
O O X 
X O . 
P2 wins
Game #755
X . O 
X X X 
O . O 
P1 wins
Game #756
O . . 
O X O 
X X X 
P1 wins
Game #757
X O O 
O X X 
O X X 
P1 wins
Game #758
. . X 
. O X 
. O X 
P1 wins
Game #759
O X O 
X O X 
O X . 
P2 wins
Game #760
X X . 
O X O 
. O X 
P1 wins
Game #761
X O X 
O . X 
. O X 
P1 wins
Game #762
O O X 
X X O 
O X X 
DRAW
Game #763
X X O 
O O O 
. X X 
P2 wins
Game #764
X O X 
X X O 
O X O 
DRAW
Game #765
X X . 
O O O 
X X O 
P2 wins
Game #766
X O . 
X . O 
X O X 
P1 wins
Game #767
O X X 
O O X 
. X O 
P2 wins
Game #768
X X O 
. X O 
X O O 
P2 wins
Game #769
O O X 
X X X 
O X O 
P1 wins
Game #770
O . X 
. X O 
X O X 
P1 wins
Game #771
X X . 
. X O 
O O X 
P1 wins
Game #772
O X X 
X O . 
X O O 
P2 wins
Game #773
X O . 
X X X 
. O O 
P1 wins
Game #774
X O O 
O X X 
X X O 
DRAW
Game #775
O X O

X O . 
P1 wins
Game #998
X X O 
O O X 
X X O 
DRAW
Game #999
. . O 
X O . 
O X X 
P2 wins
Game #1000
O . O 
X X X 
. . . 
P1 wins
P1 wins: 0.575
Draws: 0.133
P2 wins: 0.292


In [5]:
num_games = 1000

first_player_wins = 0
draws = 0
second_player_wins = 0
for i in range(1, num_games+1):
    print('Game #{}'.format(i))
    ttt = TicTacToe(4,4,4)
    outcome = ttt.play_game(random_player, random_player)
    if outcome == 1:
        first_player_wins += 1
        print('P1 wins')
    elif outcome == 0:
        draws += 1
        print('DRAW')
    else:
        second_player_wins += 1
        print('P2 wins')

print("P1 wins: {}".format(float(first_player_wins) / float(num_games)))
print("Draws: {}".format(float(draws) / float(num_games)))
print("P2 wins: {}".format(float(second_player_wins) / float(num_games)))

Game #1
X O O O 
O O X X 
O X X O 
X O X X 
DRAW
Game #2
O . X O 
O X X X 
. X X . 
O O X O 
P1 wins
Game #3
O O X O 
O O O X 
X X O X 
O X X X 
DRAW
Game #4
X O O X 
X O X O 
O O O X 
O X X X 
DRAW
Game #5
X X O O 
O O X O 
O X X X 
X O O X 
DRAW
Game #6
X O X X 
O X O O 
X O O X 
X X O O 
DRAW
Game #7
X X O X 
O X O O 
O O O X 
O X X X 
DRAW
Game #8
O O O X 
O O X X 
X X X O 
X X O . 
P1 wins
Game #9
O X X O 
O O X X 
X X X O 
O X O O 
DRAW
Game #10
O X O O 
X X O O 
X X . X 
O X O X 
P1 wins
Game #11
O X O O 
O X X O 
X O X X 
O X O X 
DRAW
Game #12
O X O O 
X X O O 
X O X X 
X O O X 
DRAW
Game #13
X X X X 
. . X O 
. O . . 
O O X O 
P1 wins
Game #14
O X O X 
X O O O 
X X X O 
O O X X 
DRAW
Game #15
O O X X 
O X O X 
O O O X 
X X X O 
DRAW
Game #16
X O O O 
O O X X 
X . O O 
X X X X 
P1 wins
Game #17
X O O X 
O X X O 
X O X O 
O X X O 
DRAW
Game #18
O O O X 
X X X O 
O O O X 
O X X X 
DRAW
Game #19
X . O X 
O X X O 
X X O O 
X . O . 
P1 wins
Game #20
X O X X 
O O O X 
X X O X 
X O O

. X O O 
O X O X 
X X . . 
O X X O 
P1 wins
Game #165
O . X O 
X O O O 
X X X X 
O O X X 
P1 wins
Game #166
. O O X 
X O X . 
O O X X 
X O X O 
P2 wins
Game #167
X X X X 
O . X O 
O X . X 
O O O . 
P1 wins
Game #168
O O X . 
O O O X 
X O X O 
X X X X 
P1 wins
Game #169
O O X O 
O X . O 
X X X X 
X X O O 
P1 wins
Game #170
X O X O 
O X X O 
X O O X 
X X O O 
DRAW
Game #171
X X O O 
O X X X 
O O X O 
O X X O 
DRAW
Game #172
X O X O 
O X O X 
O O X X 
X X O O 
DRAW
Game #173
O O X O 
O X X X 
X X X O 
. O X O 
P1 wins
Game #174
X X O O 
X X O X 
O O X O 
O . X . 
P2 wins
Game #175
X O X O 
O O X . 
X X X O 
O X X O 
P1 wins
Game #176
X O O X 
X O O X 
X X O X 
O X O O 
P2 wins
Game #177
X O X O 
X X O X 
O O X X 
X O O O 
DRAW
Game #178
X X O O 
X O X O 
O X O X 
O X O X 
DRAW
Game #179
X O O O 
O X X X 
X X O O 
X X O O 
DRAW
Game #180
O X X X 
X O O X 
O O X O 
X O O X 
DRAW
Game #181
O X X X 
X O O O 
X O O O 
O X X X 
DRAW
Game #182
X O O X 
. . O X 
O X X X 
O O . X 
P1 wins
Game #18

O X X X 
X O X O 
O O O O 
P2 wins
Game #326
X O O X 
O O O X 
X O O X 
X X . X 
P1 wins
Game #327
X O O . 
O . . O 
O X X O 
X X X X 
P1 wins
Game #328
X O X O 
X O X X 
O X X O 
O X O O 
DRAW
Game #329
X X O O 
X . O . 
X . X . 
X O . O 
P1 wins
Game #330
O X X O 
O O X O 
O . X X 
X X X O 
P1 wins
Game #331
O X X O 
X O . X 
X . X X 
O O O O 
P2 wins
Game #332
. X O X 
. O O X 
O X O O 
X X O X 
P2 wins
Game #333
O O X O 
O X X X 
X X O X 
O O X O 
DRAW
Game #334
O O X X 
X X O . 
O O O O 
X X . X 
P2 wins
Game #335
O O O X 
O O X X 
X O X O 
O X X X 
DRAW
Game #336
. X X X 
. X X O 
O O O O 
X O X O 
P2 wins
Game #337
X X X O 
O O X O 
X O O X 
O X O X 
DRAW
Game #338
O X X X 
O X X O 
X O O O 
O O X X 
DRAW
Game #339
O O . X 
O X O X 
O O X X 
O X X . 
P2 wins
Game #340
X O X O 
X O X O 
O O X X 
O X O X 
DRAW
Game #341
X X O O 
O O X X 
X X X O 
O O O X 
DRAW
Game #342
O X O X 
X O X X 
X O O O 
X O O X 
DRAW
Game #343
X O X O 
O X X . 
O X X O 
X O O X 
P1 wins
Game #344
O X O X

O O X O 
X O X X 
O . O O 
X X X X 
P1 wins
Game #490
. X . O 
. X X O 
O X X O 
. . . O 
P2 wins
Game #491
O O X O 
X . X O 
O O X O 
X X X X 
P1 wins
Game #492
O O O X 
X . O X 
. X O X 
. O X X 
P1 wins
Game #493
O X X X 
X O O O 
X O X O 
X O X O 
DRAW
Game #494
O O O X 
X O O X 
. . . X 
. . X X 
P1 wins
Game #495
X X . O 
O X O O 
O . O . 
X X X X 
P1 wins
Game #496
O O X . 
X . X O 
. . X . 
O O X X 
P1 wins
Game #497
X X X O 
X O O O 
X X O O 
O X X O 
P2 wins
Game #498
X X O X 
X . O O 
X X . . 
X O O O 
P1 wins
Game #499
. O X X 
X O X O 
O X O X 
X O X O 
P1 wins
Game #500
X O O X 
X X O X 
O X O O 
O O X X 
DRAW
Game #501
O X X X 
O O X O 
X O X . 
O X X O 
P1 wins
Game #502
X X O O 
O O O X 
X O X X 
X O X O 
DRAW
Game #503
X X O O 
O O X X 
O X X X 
X O O O 
DRAW
Game #504
O O X X 
X X O X 
O X O O 
O O X X 
DRAW
Game #505
X O O . 
X O O . 
X O X O 
X X X . 
P1 wins
Game #506
X . O O 
X O . O 
X . . . 
X . X . 
P1 wins
Game #507
O . . . 
O O X O 
X O X O 
X X X X 
P1 wins

O . X X 
O X X O 
X X O O 
X X O O 
P1 wins
Game #652
O X O X 
X X X O 
O O X X 
O X O O 
DRAW
Game #653
X O O X 
O X . O 
X O X O 
X X O X 
P1 wins
Game #654
X O X O 
X O X O 
X X O X 
O O X O 
DRAW
Game #655
O O X O 
X O X X 
O O . O 
X X X X 
P1 wins
Game #656
O X O . 
O X X X 
O X X O 
O . O X 
P2 wins
Game #657
X X X X 
O X O X 
X O X O 
O . O O 
P1 wins
Game #658
O X X X 
O X O O 
O X X O 
X O X O 
DRAW
Game #659
X O X O 
X O O O 
X O X O 
O X X X 
P2 wins
Game #660
X . O X 
O X X X 
O X O O 
X O X O 
P1 wins
Game #661
O X X X 
O X X O 
X X O O 
O O X O 
DRAW
Game #662
. O X X 
O O O X 
O . . X 
. X . X 
P1 wins
Game #663
O X O X 
X O O X 
X O O O 
X X X O 
P2 wins
Game #664
X O X X 
O O X X 
O X O X 
O X O O 
DRAW
Game #665
O O O X 
X X X O 
X O X O 
O O X X 
DRAW
Game #666
O X O O 
X X X X 
X O O . 
X X O O 
P1 wins
Game #667
X O O X 
X . . . 
X O X O 
X O . . 
P1 wins
Game #668
O O O . 
. X . X 
O . . O 
X X X X 
P1 wins
Game #669
O O X O 
X O O O 
X O X X 
X X O X 
DRAW
Game 

. X O X 
. X X X 
O O O O 
. O . X 
P2 wins
Game #823
O X . X 
O O X O 
. X . X 
X O . . 
P1 wins
Game #824
X O X O 
X X O O 
O O O X 
O X X X 
P2 wins
Game #825
X X X O 
X X O X 
O O O X 
O O X O 
P2 wins
Game #826
X X X X 
O O O X 
O X O O 
X O X . 
P1 wins
Game #827
O O X X 
O O X O 
X X O O 
O X X X 
DRAW
Game #828
O O O X 
X X O O 
X X X O 
X O O X 
DRAW
Game #829
O X X O 
O O X X 
X O O O 
X X O X 
DRAW
Game #830
O O O X 
X O X X 
O X X O 
X . O X 
P1 wins
Game #831
X X O . 
. X O X 
. X . O 
O X . O 
P1 wins
Game #832
X X O O 
O X X X 
O O O X 
X X O O 
DRAW
Game #833
O O O X 
X O X O 
O O X X 
X X O X 
DRAW
Game #834
O X X O 
O X O X 
X X O X 
O O X O 
DRAW
Game #835
O O X O 
O O X X 
X O O X 
X X X O 
P2 wins
Game #836
O X O O 
X O X O 
X O X X 
X X O O 
DRAW
Game #837
X O O X 
X X . . 
O X X X 
O O O O 
P2 wins
Game #838
O X O X 
. O X O 
X O O X 
X X . O 
P2 wins
Game #839
X X O . 
O X X X 
O X O O 
X X O O 
P1 wins
Game #840
X O X O 
X O O . 
O O . X 
X O X X 
P2 wins
Game 

Game #997
X X . O 
O O O O 
X O X X 
X . O X 
P2 wins
Game #998
O O X X 
X X O O 
O O X X 
X O O X 
DRAW
Game #999
O X O X 
O X X X 
X O O X 
O X O O 
DRAW
Game #1000
X . X O 
X X O O 
X O O X 
X O X O 
P1 wins
P1 wins: 0.328
Draws: 0.4
P2 wins: 0.272


---
Moving from a 3x3 board to a 4x4 board, we see player1's win rate substantially decrease, the draw rate substantially increase, and player2's win rate stays roughly the same. The state space for the 3x3 board is given by $3^9$ because we have 9 spaces for three different characters: `X`, `O`, and `empty`. We have $2*3 + 2$ ways to win on our 3x3 board.

Similarly the state space for a 4x4 board is given by $3^{16}$ so our state space drastically increases, while we'd only have an additional two ways to win. Although there are many states that can be achieved with a win, I think this shows that as the board size increases, our state space increases drastically while the number of ways to win only increases by 2.

---

<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [6]:
def alphabeta_player(game):
    return alphabeta_search(game)

def alphabeta_search(game):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''  

    # Functions used by alphabeta
    #result(self, move, state)
    def max_value(state, alpha, beta):
        # terminal state if there is a winner or there are no moves left
        # return the state value and the number of moves to get to this state
#         print((1 / np.abs(len(game.state.moves) - len(state.moves))))
        if state.utility != 0 or len(state.moves) == 0:
            return state.utility * (1 / np.abs(len(game.state.moves) - len(state.moves)))
        
        # initialize v as negative infinity
        v = float('-inf')
        
        # for the remaining moves in the given state
        for move in state.moves:
            v = max(v, min_value(game.result(move, state), alpha, beta))
            if v >= beta:
                return v * (1 / np.abs(len(game.state.moves) - len(state.moves)))  
            alpha = max(alpha, v)
#         print((v, len(game.state.moves) - len(state.moves)))
        return v * (1 / np.abs(len(game.state.moves) - len(state.moves)))

    def min_value(state, alpha, beta):
        # terminal state if there is a winner or there are no moves left
            # return the state value and the number of moves to get to this state
#         print((1 / np.abs(len(game.state.moves) - len(state.moves))))   
        if state.utility != 0 or len(state.moves) == 0:
            return state.utility * (1 / np.abs(len(game.state.moves) - len(state.moves)))
        
        # initialize v as negative infinity
        v = float('inf')
        
        # for the remaining moves in the given state
        for move in state.moves:
            v = min(v, max_value(game.result(move, state), alpha, beta))
                
            if v <= alpha:
                return v * (1 / np.abs(len(game.state.moves) - len(state.moves)))
            beta = min(beta, v)
#         print((v, len(game.state.moves) - len(state.moves)))
        return v * (1 / np.abs(len(game.state.moves) - len(state.moves)))
    
    # Body of alphabeta_cutoff_search:
    possible_actions = game.state.moves
    if (game.ncol * game.nrow) == len(possible_actions):
        rand_int = np.random.randint(low=0, high=len(possible_actions))
#         print(possible_actions[rand_int])
        return possible_actions[rand_int]
    else:
        vals = []
        for move in game.state.moves:
            v = max_value(game.result(move, game.state), float('-inf'), float('inf'))
            vals.append(v)
#         print(vals)
        m = max(vals)
        l = [i for i, j in enumerate(vals) if j == m]
        return game.state.moves[l[0]]


    
'''
I just couldn't get this to run properly. Perhaps on my own without the constraints of trying with the current tic-tac-toe implementation. I wish I had more time to try, but the algorithm's power is definitely witnessed in the problem

'''

"\nI just couldn't get this to run properly. Perhaps on my own without the constraints of trying with the current tic-tac-toe implementation. I wish I had more time to try, but the algorithm's power is definitely witnessed in the problem\n\n"

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [7]:
# Your code here.
num_games = 10

first_player_wins1 = 0
draws1 = 0
second_player_wins1 = 0
for i in range(1, num_games+1):
    print('Game #{}'.format(i))
    ttt = TicTacToe(3,3,3)
    outcome = ttt.play_game(alphabeta_player, random_player)
    if outcome == 1:
        first_player_wins1 += 1
        print('P1 wins')
    elif outcome == 0:
        draws1 += 1
        print('DRAW')
    else:
        second_player_wins1 += 1
        print('P2 wins')

print("alpha-beta vs. random")
print("P1 wins: {}".format(float(first_player_wins1) / float(num_games)))
print("Draws: {}".format(float(draws1) / float(num_games)))
print("P2 wins: {}".format(float(second_player_wins1) / float(num_games)))

first_player_wins2 = 0
draws2 = 0
second_player_wins2 = 0
for i in range(1, num_games+1):
    print('Game #{}'.format(i))
    ttt = TicTacToe(3,3,3)
    outcome = ttt.play_game(random_player, alphabeta_player)
    if outcome == 1:
        first_player_wins2 += 1
        print('P1 wins')
    elif outcome == 0:
        draws2 += 1
        print('DRAW')
    else:
        second_player_wins2 += 1
        print('P2 wins')

print("random vs. alpha-beta")
print("P1 wins: {}".format(float(first_player_wins2) / float(num_games)))
print("Draws: {}".format(float(draws2) / float(num_games)))
print("P2 wins: {}".format(float(second_player_wins2) / float(num_games)))

first_player_wins3 = 0
draws3 = 0
second_player_wins3 = 0
for i in range(1, num_games+1):
    print('Game #{}'.format(i))
    ttt = TicTacToe(3,3,3)
    outcome = ttt.play_game(alphabeta_player, alphabeta_player)
    if outcome == 1:
        first_player_wins3 += 1
        print('P1 wins')
    elif outcome == 0:
        draws3 += 1
        print('DRAW')
    else:
        second_player_wins3 += 1
        print('P2 wins')
print("alpha-beta vs. alpha-beta")
print("P1 wins: {}".format(float(first_player_wins3) / float(num_games)))
print("Draws: {}".format(float(draws3) / float(num_games)))
print("P2 wins: {}".format(float(second_player_wins3) / float(num_games)))


Game #1
O X X 
. . X 
O O X 
P1 wins
Game #2
X . O 
. X . 
O . X 
P1 wins
Game #3
X X X 
. O O 
. . . 
P1 wins
Game #4
X . O 
X X X 
O O . 
P1 wins
Game #5
X X O 
X O O 
X . . 
P1 wins
Game #6
X X X 
. O . 
O . . 
P1 wins
Game #7
X X X 
. O . 
. O . 
P1 wins
Game #8
X X X 
O . O 
. X O 
P1 wins
Game #9
X O X 
X . . 
X O O 
P1 wins
Game #10
X O X 
X O X 
O X O 
DRAW
alpha-beta vs. random
P1 wins: 0.9
Draws: 0.1
P2 wins: 0.0
Game #1
O O X 
X O X 
O X X 
P1 wins
Game #2
O O X 
X X X 
X O O 
P1 wins
Game #3
O O . 
O X . 
X X X 
P1 wins
Game #4
O O X 
O X . 
X . X 
P1 wins
Game #5
O O X 
. . X 
. . X 
P1 wins
Game #6
O O X 
O X X 
X O X 
P1 wins
Game #7
O O O 
X O X 
X . X 
P2 wins
Game #8
O O X 
X X O 
X O X 
P1 wins
Game #9
X O O 
X O . 
X . X 
P1 wins
Game #10
O O O 
X X O 
X X . 
P2 wins
random vs. alpha-beta
P1 wins: 0.8
Draws: 0.0
P2 wins: 0.2
Game #1
X O X 
O X O 
X . . 
P1 wins
Game #2
X X X 
O O . 
. . . 
P1 wins
Game #3
O X O 
O X . 
X X . 
P1 wins
Game #4
O X O 
. X . 
. X . 
P1 

---
I couldn't quite get it to run properly. I tried to implement a system to prioritize moves that win in fewer moves but it appears I didn't implement the algorithm correctly in the first place.

---

<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [8]:
three_by_three = 3*3
no_pruning = 3**(three_by_three)


print("max # of states expanded without pruning: {}".format(no_pruning))

max # of states expanded without pruning: 19683


---

There is a maximum of `19,683` states expanded without pruning using the minimax algorithm because every state is expanded.

I kinda ran out of time knowing how many nodes we'd expand with pruning, but I believe pruning brings the number down substantially enough where it becomes feasible for tic-tac-toe. On chess we'd need to limit the depth of our search.

--- 

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

In [9]:
for i in range(10):
    print((1 - (1-i)-(i-1)) - 1)

0
0
0
0
0
0
0
0
0
0


$L(d,x) = (1 - (1-x)-(d-1))$

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}


### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [10]:
target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [11]:
def recipe_success(state):
    fitness = 0

    # problem specific
    # kinda cheating having this in here idk...
    target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

    i = 0
    for gs, gt in zip(state, target): 
        if gs != gt: 
    ### 1           
    #           fitness+=1
    ### 2           
    #           x = np.abs(gs - gt)
    #           if x < 10: fitness += 1
    #           elif x < 20: fitness += 2
    #           elif x < 30: fitness += 3
    #           else: fitness += 5
    ### 3           
    #           fitness += 100 * np.abs((gs/108) - (gt/108))
    ### 4: Also really good
    #           fitness += 100 * np.abs((gs/target[i]) - (gt/target[i]))
    #           i+=1
    ### 5
    #           x = np.abs((gs/target[i]) - (gt/target[i]))
    #           if x < 0.1: fitness += 1
    #           elif x < 0.2: fitness += 2
    #           elif x < 0.3: fitness += 3
    #           elif x < 0.4: fitness += 4
    #           elif x < 0.5: fitness += 5
    #           elif x < 0.6: fitness += 6
    #           elif x < 0.7: fitness += 7
    #           elif x < 0.8: fitness += 8
    #           elif x < 0.9: fitness += 9
    #           else: fitness += 10
    #           i+=1
    ### 6: MY BEST ONE
            x = np.abs((gs/target[i]) - (gt/target[i]))
            if x < 0.05: fitness += 1
            elif x < 0.1: fitness += 2
            elif x < 0.15: fitness += 3
            elif x < 0.2: fitness += 4
            elif x < 0.25: fitness += 5
            elif x < 0.3: fitness += 6
            elif x < 0.35: fitness += 7
            elif x < 0.4: fitness += 8
            elif x < 0.45: fitness += 9
            elif x < 0.5: fitness += 10
            elif x < 0.55: fitness += 11
            elif x < 0.6: fitness += 12
            elif x < 0.65: fitness += 13
            elif x < 0.7: fitness += 14
            elif x < 0.75: fitness += 15
            elif x < 0.8: fitness += 16
            elif x < 0.85: fitness += 17
            elif x < 0.9: fitness += 18
            elif x < 0.95: fitness += 19
            else: fitness += 20
            i+=1
    return fitness

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [12]:
'''
adapted from GeeksForGeeks and class notebook codes
'''
class problem:

    def __init__(self, dna_length, population_length, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)

        In our case, I've chosen to minimize this fitness_goal at 0
        This felt natural for numerous reasons and I cannot necessarily think of an example when this would be helpful to maximize
        '''
        self.n_pop = population_length
        self.n_dna = dna_length
        self.initial_population = self.create_gnome()
        self.population = self.initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.fitness_goal = fitness_goal

    def mutate_gene(self, gene=None): 
        ''' 
        create random genes for mutation 

        problem specific: 
        (really only works on this problem with lists of ints)
        lots of trial and tribulation here...
        ''' 

        # initialization: gene=None 
        # return a gene in the valid range for the problem
        if gene == None:
            return np.random.choice(np.arange(0, 108.5, 0.5))


        # else 

        # choose if the gene will be randomly mutated (sorta aids like a random restart in hillclimbing)
        random_mutate = np.random.choice([True, False], p=[self.p_mutate, 1-self.p_mutate])

        # randomly mutate if chosen
        if random_mutate:
            return np.random.choice(np.arange(0, 108.5, 0.5))

        # else

        # choose if the plus minus will be high or low	
        high_pm = np.random.choice([True, False], p=[self.p_mutate, 1-self.p_mutate])

        # we choose our plus minus 	
        if high_pm:
            plus_minus = 2.5
        else:
            plus_minus = 1

        # get random value x in range [-plus_minus, plus_minus] such that adding the value x to gene doesn't invalidate it
        x = 0
        while x == 0:
            x = np.random.choice(np.arange(-plus_minus, plus_minus+0.5, 0.5))
            if (gene + x < 0) or (gene + x > 108.5):
                x = 0
        # finally return said value x with the gene
        return gene + x


    def create_gnome(self): 
        ''' 
        create chromosome or string of genes 
        '''
        # very straightforward, creating the gnome, very problem specific
        population = []
        for _ in range(self.n_pop):
            population.append([self.mutate_gene() for _ in range(self.n_dna)])
        return population 

    def mate(self, par1, par2): 
        ''' 
        Perform mating and produce new offspring 
        '''
        # chromosome for offspring 
        child_chromosome = [] 

        # computing probabilties for each parents gene being chosen given the mutation probability
        p2_prob = 1 - self.p_mutate
        p1_prob = p2_prob / 2

        for gp1, gp2 in zip(par1, par2):     

                # random probability   
                prob = np.random.random() 
                p2_prob = 1 - self.p_mutate
                p1_prob = p2_prob / 2

                # if prob is less than 0.45, insert gene 
                # from parent 1  
                if prob < p1_prob: 
                    child_chromosome.append(gp1) 

                # if prob is between 0.45 and 0.90, insert 
                # gene from parent 2 
                elif prob < p2_prob: 
                    child_chromosome.append(gp2) 

                # otherwise insert random gene(mutate),  
                # for maintaining diversity 
                else: 
                    child_chromosome.append(self.mutate_gene(int((gp1 + gp2) / 2)))  
        # generated chromosome for offspring 
        return child_chromosome

In [13]:
# taken from GeeksForGeeks and adapted to this problem...
# gave me some better results than the class notebook
def genetic_algorithm(problem, n_iter):

    generation = 1
    pop = problem.population

    for _ in range(n_iter-1):

        # sort the population in increasing order of fitness score 
        pop = sorted(pop, key = lambda x: problem.objective_function(x)) 

        # if the individual having lowest fitness score ie.  
        # 0 then we know that we have reached to the target 
        # and break the loop 
        if problem.objective_function(pop[0]) <= 0: 
            print("\nMATCH FOUND\n!!!Generation: {}\tString: {}\tFitness: {}".format(generation, pop[0], problem.objective_function(pop[0])))
            return

        # Otherwise generate new offsprings for new generation 
        new_generation = [] 

        for _ in range(problem.n_pop):
            h = int(len(pop) / 2) 
            indices = np.arange(h)
            rnd_indices = np.random.choice(h, size=2)
            child = problem.mate(pop[-rnd_indices[0]], pop[-rnd_indices[1]]) 
            new_generation.append(child) 


        new_generation = sorted(new_generation, key = lambda x: problem.objective_function(x))

        s = int((10*problem.n_pop)/100) 
        new_generation = new_generation[:s]

        # Perform Elitism, that mean 10% of fittest population 
        # goes to the next generation 
        new_generation += pop[:s]

        pop = sorted(new_generation, key = lambda x: problem.objective_function(x))

        print("Generation: {}\tString: {}\tFitness: {}".format(generation, pop[0], problem.objective_function(pop[0]))) 

        generation += 1


    print("\nMATCH NOT FOUND...\nGeneration: {}\tString: {}\tFitness: {}".format(generation, pop[0], problem.objective_function(pop[0])))


In [14]:
'''
Original problem
'''
genetic_problem = problem(dna_length=9,
                          population_length=10, 
                          fitness_goal=0, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 200)
print(out)

Generation: 1	String: [23.0, 59.5, 45.0, 78.0, 47.5, 71.5, 34.0, 89.0, 88.0]	Fitness: 113
Generation: 2	String: [23.0, 59.5, 44.5, 78.0, 47.5, 71.5, 34.0, 89.0, 87.5]	Fitness: 113
Generation: 3	String: [23.5, 59.5, 44.5, 78.0, 47.5, 71.5, 34.0, 89.0, 87.5]	Fitness: 112
Generation: 4	String: [23.5, 59.5, 44.5, 78.0, 47.5, 71.5, 34.0, 89.0, 87.5]	Fitness: 112
Generation: 5	String: [23.5, 58.0, 44.5, 77.0, 46.0, 71.5, 34.0, 89.0, 87.5]	Fitness: 111
Generation: 6	String: [23.5, 58.0, 44.5, 77.0, 46.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 111
Generation: 7	String: [23.5, 57.5, 44.5, 77.0, 47.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 110
Generation: 8	String: [23.5, 57.5, 44.5, 77.0, 47.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 110
Generation: 9	String: [23.5, 57.5, 44.5, 77.0, 47.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 110
Generation: 10	String: [24.0, 57.5, 44.5, 77.0, 47.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 110
Generation: 11	String: [24.0, 57.5, 43.5, 77.0, 47.0, 71.5, 34.0, 88.0, 87.5]	Fitness: 110
Generati

Generation: 99	String: [39.0, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 87.5]	Fitness: 88
Generation: 100	String: [39.0, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 87.5]	Fitness: 88
Generation: 101	String: [39.0, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 87.5]	Fitness: 88
Generation: 102	String: [39.0, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 87.5]	Fitness: 88
Generation: 103	String: [38.0, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 87.5]	Fitness: 88
Generation: 104	String: [37.5, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 88.0]	Fitness: 87
Generation: 105	String: [37.5, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 88.0]	Fitness: 87
Generation: 106	String: [37.5, 35.5, 46.5, 57.5, 63.5, 97.5, 11.0, 76.0, 88.0]	Fitness: 87
Generation: 107	String: [37.5, 35.5, 46.5, 57.5, 63.5, 103.5, 11.0, 76.0, 88.0]	Fitness: 86
Generation: 108	String: [37.5, 35.5, 46.5, 57.5, 63.5, 103.5, 11.0, 76.0, 88.0]	Fitness: 86
Generation: 109	String: [37.5, 35.5, 46.5, 57.5, 63.5, 103.5, 11.0, 76.0, 88.0]	Fitness: 

In [15]:
'''
Original problem with 500 iterations to allow my program to finish
'''
genetic_problem = problem(dna_length=9,
                          population_length=10, 
                          fitness_goal=0, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 500)
print(out)

Generation: 1	String: [58.0, 37.5, 52.5, 40.0, 3.5, 97.0, 30.5, 64.0, 73.0]	Fitness: 104
Generation: 2	String: [57.0, 37.5, 52.5, 40.0, 3.5, 97.0, 30.5, 64.0, 73.0]	Fitness: 103
Generation: 3	String: [57.0, 37.5, 52.5, 40.0, 3.5, 97.0, 30.5, 64.0, 73.0]	Fitness: 103
Generation: 4	String: [57.0, 37.5, 52.5, 40.0, 2.0, 98.0, 30.5, 64.0, 73.0]	Fitness: 102
Generation: 5	String: [57.0, 37.5, 52.5, 40.0, 1.0, 98.0, 30.5, 64.0, 73.0]	Fitness: 101
Generation: 6	String: [57.0, 37.5, 52.5, 40.0, 1.0, 98.0, 30.5, 63.0, 73.0]	Fitness: 101
Generation: 7	String: [57.0, 37.5, 52.5, 40.0, 1.0, 98.5, 30.5, 63.0, 73.0]	Fitness: 101
Generation: 8	String: [57.0, 37.5, 52.5, 40.0, 1.0, 98.5, 30.5, 63.0, 73.0]	Fitness: 101
Generation: 9	String: [57.0, 37.5, 52.5, 40.0, 0.5, 98.5, 30.5, 63.0, 73.0]	Fitness: 93
Generation: 10	String: [57.0, 37.5, 52.5, 40.0, 0.5, 98.5, 30.5, 63.0, 73.0]	Fitness: 93
Generation: 11	String: [57.0, 37.5, 52.5, 40.0, 0.5, 98.5, 30.5, 63.0, 73.0]	Fitness: 93
Generation: 12	String:

Generation: 95	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.5, 100.0]	Fitness: 57
Generation: 96	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.5, 100.0]	Fitness: 57
Generation: 97	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.5, 100.0]	Fitness: 57
Generation: 98	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.5, 100.0]	Fitness: 57
Generation: 99	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 60.0, 100.0]	Fitness: 57
Generation: 100	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.0, 100.0]	Fitness: 57
Generation: 101	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.0, 99.5]	Fitness: 57
Generation: 102	String: [32.5, 36.0, 47.5, 31.0, 1.0, 96.0, 1.0, 59.0, 99.5]	Fitness: 57
Generation: 103	String: [32.5, 36.0, 47.5, 31.0, 1.0, 95.5, 1.0, 59.0, 99.5]	Fitness: 57
Generation: 104	String: [32.5, 36.0, 47.5, 31.0, 1.0, 95.5, 1.0, 59.0, 99.5]	Fitness: 57
Generation: 105	String: [32.5, 36.0, 47.5, 31.0, 1.0, 95.5, 1.0, 59.0, 99.5]	Fitness: 57
Generation: 106	Stri

Generation: 212	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 213	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 214	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 215	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 216	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 217	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 218	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 219	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 220	String: [36.0, 36.0, 47.5, 1.5, 1.0, 87.5, 1.0, 0.5, 96.0]	Fitness: 11
Generation: 221	String: [36.0, 36.0, 48.0, 1.5, 1.0, 98.0, 1.0, 0.5, 96.0]	Fitness: 7
Generation: 222	String: [36.0, 36.0, 48.0, 0.5, 1.0, 98.0, 1.0, 0.5, 96.0]	Fitness: 7
Generation: 223	String: [36.0, 36.0, 48.0, 0.

Generation: 311	String: [35.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 312	String: [35.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 313	String: [35.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 314	String: [35.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 315	String: [36.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 316	String: [36.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 317	String: [36.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 318	String: [36.0, 36.0, 46.5, 1.0, 1.0, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 319	String: [36.0, 36.0, 46.5, 1.0, 1.5, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 320	String: [36.0, 36.0, 46.5, 1.0, 1.5, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 321	String: [36.0, 36.0, 46.5, 1.0, 1.5, 103.5, 1.0, 0.5, 96.0]	Fitness: 4
Generation: 322	String: [36.0, 36.0, 46.5, 

Generation: 412	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 413	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 414	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 415	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 416	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 417	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 418	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 419	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 420	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 421	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 422	String: [36.0, 36.0, 48.0, 1.0, 1.0, 106.5, 1.0, 0.5, 96.0]	Fitness: 1
Generation: 423	String: [36.0, 36.0, 48.0, 

In [16]:
'''
Original problem with:

population size = 100
'''
genetic_problem = problem(dna_length=9,
                          population_length=100, 
                          fitness_goal=0, 
                          mutation_probability=0.1, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 500)
print(out)

Generation: 1	String: [35.0, 33.0, 68.0, 61.5, 36.5, 29.0, 103.5, 96.5, 89.0]	Fitness: 109
Generation: 2	String: [35.0, 33.0, 68.0, 92.0, 36.5, 106.5, 103.5, 96.5, 89.0]	Fitness: 95
Generation: 3	String: [35.0, 33.0, 68.0, 92.0, 36.5, 106.5, 103.5, 96.5, 89.0]	Fitness: 95
Generation: 4	String: [35.0, 33.0, 40.5, 61.5, 36.5, 93.0, 12.0, 25.0, 89.0]	Fitness: 92
Generation: 5	String: [37.0, 33.0, 48.5, 92.0, 79.0, 107.5, 72.5, 96.5, 101.5]	Fitness: 87
Generation: 6	String: [37.0, 33.0, 48.5, 1.5, 79.0, 107.5, 72.5, 96.5, 87.5]	Fitness: 78
Generation: 7	String: [37.0, 33.5, 48.5, 1.5, 79.0, 107.5, 72.5, 95.0, 87.5]	Fitness: 78
Generation: 8	String: [35.0, 32.5, 48.5, 1.5, 17.0, 107.5, 90.5, 87.0, 101.5]	Fitness: 78
Generation: 9	String: [35.0, 32.5, 48.5, 1.5, 33.5, 107.5, 72.5, 87.0, 96.0]	Fitness: 76
Generation: 10	String: [35.0, 32.5, 48.5, 1.5, 33.5, 106.0, 72.5, 87.0, 96.0]	Fitness: 76
Generation: 11	String: [36.0, 33.0, 48.5, 1.5, 33.5, 107.5, 72.5, 87.0, 96.0]	Fitness: 69
Generation

In [17]:
'''
Original problem with:

mutation probability = 0.2
'''
genetic_problem = problem(dna_length=9,
                          population_length=10, 
                          fitness_goal=0, 
                          mutation_probability=0.2, 
                          objective_function=recipe_success)
out = genetic_algorithm(genetic_problem, 500)
print(out)

Generation: 1	String: [33.5, 37.5, 43.5, 81.0, 38.0, 98.5, 83.0, 48.5, 79.5]	Fitness: 91
Generation: 2	String: [35.0, 37.5, 43.5, 81.0, 39.0, 98.5, 84.0, 48.5, 81.5]	Fitness: 90
Generation: 3	String: [35.0, 37.5, 43.5, 80.0, 39.0, 98.5, 84.0, 48.5, 81.5]	Fitness: 90
Generation: 4	String: [35.0, 37.5, 43.5, 80.0, 39.0, 98.5, 84.0, 17.0, 81.5]	Fitness: 90
Generation: 5	String: [35.0, 37.5, 43.5, 80.0, 39.0, 98.5, 84.0, 17.0, 81.5]	Fitness: 90
Generation: 6	String: [35.5, 37.5, 43.5, 80.0, 38.5, 98.5, 84.0, 16.0, 82.0]	Fitness: 89
Generation: 7	String: [35.5, 37.5, 43.5, 80.0, 38.5, 98.5, 81.5, 16.0, 82.0]	Fitness: 89
Generation: 8	String: [35.5, 37.5, 43.5, 80.0, 38.5, 98.5, 81.5, 16.0, 82.0]	Fitness: 89
Generation: 9	String: [35.5, 37.5, 43.5, 80.0, 38.5, 98.5, 81.5, 16.0, 82.0]	Fitness: 89
Generation: 10	String: [35.5, 37.5, 43.5, 81.0, 38.5, 97.5, 81.5, 16.0, 82.0]	Fitness: 89
Generation: 11	String: [35.5, 37.5, 43.5, 81.0, 38.5, 106.5, 81.5, 16.0, 82.0]	Fitness: 88
Generation: 12	Str

Generation: 100	String: [36.0, 37.0, 47.5, 5.0, 107.5, 93.5, 18.5, 57.0, 96.0]	Fitness: 68
Generation: 101	String: [36.0, 37.0, 47.5, 5.0, 106.0, 93.5, 15.5, 57.0, 96.0]	Fitness: 67
Generation: 102	String: [36.0, 37.0, 47.5, 5.0, 106.0, 93.5, 15.5, 56.0, 96.0]	Fitness: 67
Generation: 103	String: [36.0, 37.0, 47.5, 5.0, 2.0, 94.0, 15.5, 56.0, 96.0]	Fitness: 67
Generation: 104	String: [36.0, 37.0, 47.5, 5.0, 1.0, 94.0, 15.5, 56.0, 96.0]	Fitness: 55
Generation: 105	String: [36.0, 37.0, 47.5, 5.0, 1.0, 94.0, 15.5, 57.0, 96.0]	Fitness: 55
Generation: 106	String: [36.0, 37.0, 47.5, 5.0, 1.0, 17.5, 15.5, 57.0, 96.0]	Fitness: 55
Generation: 107	String: [36.0, 37.0, 47.5, 5.0, 1.0, 96.0, 15.5, 57.0, 96.0]	Fitness: 55
Generation: 108	String: [36.0, 36.5, 47.5, 5.0, 1.0, 96.0, 15.5, 57.0, 96.0]	Fitness: 55
Generation: 109	String: [36.0, 36.5, 48.5, 4.5, 1.0, 95.5, 15.5, 57.0, 96.0]	Fitness: 55
Generation: 110	String: [36.0, 36.5, 48.5, 4.5, 1.0, 94.0, 14.5, 57.0, 96.0]	Fitness: 55
Generation: 111

Generation: 208	String: [36.0, 36.0, 49.5, 2.0, 3.0, 45.0, 78.5, 1.5, 96.0]	Fitness: 44
Generation: 209	String: [36.0, 36.0, 49.5, 2.0, 3.0, 45.0, 78.5, 1.5, 96.0]	Fitness: 44
Generation: 210	String: [36.0, 36.0, 49.5, 2.0, 3.0, 45.0, 78.5, 0.5, 96.0]	Fitness: 43
Generation: 211	String: [36.0, 36.0, 49.5, 2.0, 3.0, 45.0, 78.5, 0.5, 96.0]	Fitness: 43
Generation: 212	String: [36.0, 36.0, 49.5, 1.0, 3.0, 45.0, 78.5, 0.5, 96.0]	Fitness: 43
Generation: 213	String: [36.0, 36.0, 49.5, 1.0, 3.0, 45.0, 78.5, 0.5, 96.0]	Fitness: 43
Generation: 214	String: [36.0, 36.0, 49.5, 1.0, 1.5, 45.0, 78.5, 0.5, 96.0]	Fitness: 42
Generation: 215	String: [36.0, 36.0, 49.5, 1.0, 1.5, 45.0, 78.5, 0.5, 96.0]	Fitness: 42
Generation: 216	String: [36.0, 36.0, 48.0, 1.0, 0.0, 45.0, 78.5, 0.5, 96.0]	Fitness: 41
Generation: 217	String: [36.0, 36.0, 48.0, 1.0, 0.0, 45.0, 77.0, 0.5, 96.0]	Fitness: 41
Generation: 218	String: [36.0, 36.0, 48.0, 1.0, 0.5, 44.0, 77.0, 0.5, 96.0]	Fitness: 41
Generation: 219	String: [36.0, 3

### (3c)

Report the following: **old questions**
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

Report the following: **new questions**
- Report whether you minimized of maximized the objective function and whether that led to any major changes in how you designed the probability of reproduction. A couple sentences here is fine.
- How many ingredients you ended up matching. e.g. target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96], perhaps your algorithm achieved [36, 42, 8, 1, 1, 100, 56, 0.5, 0], then you would have matched 4 of the ingredient values.
- How many iterations you tried in order to get this answer. (Don't burn up your machine in the process)

---

**Old questions:**

- It depended on how good my initial population was but it typically took around 250-500 generations. Sometimes it took a little more time too though...
- A larger population size helps us achieve our goal faster because there is more variation to choose from
- More mutation also helped us achieve our goal a little faster

**New questions:**

- I minimized the function. It seemed rather obvious to try and do this, and I couldn't think of an application where we would want to model this by maximizing the function (aside from the example in class). I suppose it could exist, but for the purposes of matching a list (or a string for that matter), it would seem good to minimize the difference.
- Although my algorithm is relatively random, it usually gets at least 3-5 items correct in 200 iterations. 
- My machine got the correct answer in under 1000 iterations almost every time and in under 500 iterations quite frequently

---

In [18]:
# Your code here.